In [15]:
import pandas as pd
import numpy as np

In [16]:
data = pd.read_parquet("../data/train.parquet")
data.head()

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.121263,-0.417690,0.005399,0.125948,0.058359,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.072944,-0.436590,-0.102483,0.017551,0.007149,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.173820,-0.213489,0.096067,0.215709,0.107133,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452


In [17]:
x_data = data.iloc[:, 5:-1].copy()

In [18]:
# Clean uninformative cols
neg_inf_cols = [col for col in x_data.columns if np.isneginf(x_data[col]).all()]
x_data.drop(columns=neg_inf_cols, inplace=True)

unique_val_cols = [col for col in x_data.columns if x_data[col].nunique() == 1]
x_data.drop(columns=unique_val_cols, inplace=True)

In [19]:
x_data_with_target = x_data.copy()
x_data_with_target["label"] = data["label"]

In [20]:
import gc

del data
del x_data
gc.collect()

677

In [21]:
correlations = (
    x_data_with_target
    .drop(columns="label")
    .corrwith(x_data_with_target["label"])
    .sort_values(ascending=False)
)
print(correlations)

X21     0.069401
X20     0.067667
X28     0.064092
X863    0.064057
X29     0.062339
          ...   
X580   -0.041725
X95    -0.042948
X137   -0.044290
X524   -0.048020
X531   -0.056184
Length: 863, dtype: float64


In [26]:
top_percentile = np.percentile(x_data_with_target["label"], 95)
bot_percentile = np.percentile(x_data_with_target["label"], 5)

x_data_extreme_top = x_data_with_target[x_data_with_target["label"] > top_percentile]
x_data_extreme_bot = x_data_with_target[x_data_with_target["label"] < bot_percentile]
x_data_extreme = pd.concat([x_data_extreme_top, x_data_extreme_bot])

In [28]:
del x_data_with_target
gc.collect()

0

In [27]:
for df in [x_data_extreme_top, x_data_extreme_bot, x_data_extreme]:
    correlations = (
        df
        .drop(columns="label")
        .corrwith(df["label"])
        .sort_values(ascending=False)
    )
    print(correlations)

X849    0.181019
X205    0.178474
X729    0.176885
X181    0.176761
X175    0.176409
          ...   
X173   -0.185095
X426   -0.186230
X179   -0.187539
X420   -0.199055
X414   -0.204036
Length: 863, dtype: float64
X841    0.185548
X837    0.183585
X833    0.181078
X221    0.180494
X777    0.177421
          ...   
X594   -0.185491
X9     -0.190387
X879   -0.192173
X878   -0.216843
X877   -0.236934
Length: 863, dtype: float64
X181    0.184705
X471    0.184108
X856    0.182991
X175    0.182231
X472    0.181195
          ...   
X9     -0.170084
X137   -0.173009
X131   -0.174823
X524   -0.186248
X531   -0.206567
Length: 863, dtype: float64


## PCA

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

pca = PCA()
pca.fit(x_data)

pca_eigval = pca.explained_variance_

# Kaiser's criterion
n_components = (pca.explained_variance_ > 1).sum()
print(n_components)

113


In [56]:
print(pca.explained_variance_ratio_[:n_components].sum())

0.9001244687052725
